In [29]:
import numpy as np
import pandas as pd

#Constante de gravitación (SI)
G = 6.6743*10**(-11)
#Masa solar (kg)
MS = 1.989*10**(30)


Distancia de Hubble (para redshifts pequeños)

$d = z D_H = z \dfrac{c}{H_0} = z (9.26\times 10^{25} h^{-1} )$ m

In [18]:
def Distancia(z):
    h0 = 0.71
    #Del Carroll p.1071
    H0 = h0*3.24*10**(-18)
    DH = (299792458)/H0
    return z*DH

Posiciones angulares y velocidades relativas al centroide y a la velocidad media

$ (\alpha_{rel}, \delta_{rel}) = (\alpha_i - \alpha_{av}, \delta_{i} - \delta_{av})$

$V_{i, rel} = v_i - v_{av}$


In [19]:
def PosicionesRelativas(dir):
    datos = pd.read_csv(dir)
    
    # Posiciones            
    AR = datos.ra*(np.pi/180)    #En radianes 
    DEC = datos.dec*(np.pi/180)  #En radianes
    V = datos.vel*1000           #En m/s
    N = len(AR)
    
    #Centroide y velocidad promedio
    cent = [np.mean(AR), np.mean(DEC)]
    Vm = np.mean(V)
    
    
    #Definimos arreglos, después los llenamos acorde a las posiciones relativas
    ARrel = [ ]; DECrel = [ ]; Vrel =[]
    for i in range(N):
        ARrel.append(AR[i] - cent[0])   
        DECrel.append(DEC[i] - cent[1]) 
        Vrel.append( V[i] - Vm )

    
    return ARrel, DECrel, N, Vrel
    
    

### Estimador virial

$M_{virial} = \dfrac{3\pi N }{2 G} \dfrac{\sum_{i=1}^{N} V_{z i }^2}{\sum_{i<j} (1/ R_{\perp, i j})}$

Donde
$R_{\perp, ij} = D \tan\theta_{\perp, ij} \approx D \theta_{\perp, ij}$

y

$\theta_{ij}^2 = (\alpha_{rel,i}  - \alpha_{rel, j})^2 + (\delta_{rel, i} - \delta_{rel, j})^2$

In [20]:
#En el archivo tener encabezados 
#ra-> Ascensión recta, #dec-> declinación y #vel-> Velocidades 
def EstimadorVirial(dir, D):
    
    #Posiciones relativas, # de galacias y velocidades relativas
    ARrel, DECrel, N, Vrel = PosicionesRelativas(dir)
    
    #Diferencia angular entre galaxias
    theta = []
    for i in range(N):
        for j in range(i+1, N):
            ARdif = ARrel[i] - ARrel[j] 
            DECdif = DECrel[i] - DECrel[j]
            theta.append( 1/np.sqrt( ARdif**2 + DECdif**2)   )
            #print("(",i,",", j, ")", end='\t')
            #print(i==j, end='\t')
    
    #Sumamos el resultado
    SumTheta = sum(theta)
    
    #Suma de velocidades al cuadrado
    V2 = []
    for i in range(N):
        V2.append(Vrel[i]*Vrel[i])
    SumV2 = sum(V2)
    
    #Estimación
    M = (3*np.pi*N*D/(2*G))*(SumV2/SumTheta)
    print("MASA VIRIAL (kg): ", M)
    print("En masas solares")
    print("Masa:" , M/MS, "M⊙")
    

### Estimador de masa mediana


$M_{med} = \dfrac{f_{me}}{G} \sum_{i} \sum_{j} [ (V_{zi} - V_{zj})^2 R_{\perp, ij} ] = \dfrac{f_{me}}{G}\;  med_{ij}[ (V_{zi} - V_{zj})^2 R_{\perp, ij} ],  \qquad f_{me} = 6.5$

In [21]:
def EstimadorMediana(dir, D):
    f =6.5
    #Sacamos posiciones relativas, número de galaxias y velocidades
    ARrel, DECrel, N, Vrel = PosicionesRelativas(dir)
    
    #Diferencia de ángulos
    thetas = np.zeros((N, N))
    for i in range(N):
        for j in range(i+1, N):
            ARdif = ARrel[i] - ARrel[j]   
            DECdif = DECrel[i] - DECrel[j]
            #print(i, j, ARdif, DECdif)
            thetas[i, j] = np.sqrt( ARdif**2 + DECdif**2  )
            #print("(",i,",", j, ")", end='\t')
            #print(i==j, end='\t')
        
    #Realizamos la sumatoria
    Suma = []
    Matriz = np.zeros([N, N])
    
    for i in range(N):
        for j in range(N):
            arg = ( (Vrel[i] - Vrel[j])**2)*thetas[i, j]
            Suma.append(arg)
            Matriz[i, j] = arg
    Suma = sum(Suma)
    
    #Convertimos la matriz en un arreglo lineal
    array = Matriz.flatten()
    #Eliminamos los valores nulos y los guardamos en otro arreglo
    array2 = []
    for i in range(len(array)):
        if array[i]>0: array2.append(array[i])
    
    mediana = np.median(array2)
    
    #Estimación de masa
    M = f*Suma*D/G
    print("\nMASA MEDIANA SUMA (kg): {:.2E}".format(M))
    print("En masas solares")
    print("Masa: {:.2E}".format(M/MS))

    M = f*mediana*D/G

    print("\nMASA MEDIANA MEDIANA (kg): {:.2E}".format(M))
    print("En masas solares")
    print("Masa: {:.2E}".format(M/MS))

    
    
    

### Masa proyectada


$M_{PM} = \dfrac{f_{PM}}{G(N-1.5) } \sum_{i} V_{zi}^2 R_{\perp i}, \qquad f_{MP}= \dfrac{32}{\pi} $

In [22]:
def EstimadorProyectada(dir, D):
    #Constantes propias del estimador
    a = 1.5; f = 32/np.pi
    
    #Posiciones relativas, número de galaxias y velocidades
    ARrel, DECrel, N, Vrel = PosicionesRelativas(dir)
    
    #Sumatoria
    theta = []
    for i in range(N):
        theta.append( np.sqrt( ARrel[i]**2 + DECrel[i]**2 )  )
    
    #Suma velocidades al cuadrado
    V2 =[]
    for i in range(N):
        V2.append(Vrel[i]*Vrel[i])

    suma = []
    for i in range(N):
        suma.append(V2[i]*theta[i])
    
    Suma = sum(suma)
    
    Suma = sum( (V**2)*theta  )
    
    #Estimación
    M = (f*Suma*D)/(G*(N-a))
    print("MASA PROYECTADA (kg): ", M)
    print("En masas solares")
    print("Masa:" , M/MS, "M⊙")

    
    
        
    
    

La estimación de masa promedio está dada por:

$\displaystyle M_{av} = \frac{f_{av}}{G} \dfrac{2 D}{N(N-1)} \sum_{i<j} (V_{zi} - V_{zj})^2 \theta_{ij}, \qquad f_{av} = 2.8$

In [23]:
def EstimadorPromedio(dir, D):
    #Constante propia del estimador
    f = 2.8
    
    #Posiciones relativas, número de galaxias y velocidades
    ARrel, DECrel, N, V = PosicionesRelativas(dir)
    
    #Diferencia de ángulos
    thetas = np.zeros((N, N))
    for i in range(N):
        for j in range(i+1, N):
            ARdif = ARrel[i] - ARrel[j]   
            DECdif = DECrel[i] - DECrel[j]
            #print(i, j, ARdif, DECdif)
            thetas[i, j] = np.sqrt( ARdif**2 + DECdif**2  )
            #print("(",i,",", j, ")", end='\t')
            #print(i==j, end='\t')
    
    #Realizamos la sumatoria
    Sum = []
    for i in range(N):
        for j in range(i+1, N):
            #print("(",i, j,")",  i<j)
            Sum.append(  ((V[i] - V[j])**2)*thetas[i, j] )
            #print("[V[",i,"] + V[", j,"]]R[", i, ",", j,"]") 
    Suma = sum(Sum)
            
    #Estimación
    M =  (f*2*D*Suma)/(G*N*(N-1))
    print("MASA PROMEDIO (kg): ", M)
    print("En masas solares")
    print("Masa:" , M/MS, "M⊙")
    

In [24]:
dirr ="Datos\A1066_dccoutcut.csv"
d = Distancia(0.0690)

In [25]:
EstimadorVirial(dirr, d)

MASA VIRIAL (kg):  1.5401685653689373e+42
En masas solares
Masa: 774343170120.1293 M⊙


In [33]:
(4.53e14)*(1.9891e30)

9.010623e+44